In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import re
import tensorflow as tf
import math
import matplotlib.cm as cm

2025-04-28 15:17:53.298141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745853473.308489  985717 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745853473.311651  985717 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 15:17:53.323162: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Extract Data

In [2]:
class DataExtract:
    def __init__(self, path_dir, n_channels, n_samples, arr_geometry):
        # private variable
        # path to data
        self._path_dir = path_dir
        
        # number of channels in the array
        self._n_channels = n_channels
        
        # number of samples of the signal
        self._n_signal_samples = n_samples

        # number of data samples
        self.n_data_samples = None

        # array geometry
        self.arr_geometry = arr_geometry
        
        # list of raw signal file names
        self._files_raw = None
        
        # list of signal information file names
        self._files_dat = None

        # Sample Frequency
        self.sampleFreq = None
        
        # Space and time of the wave
        # Measured pressure at mirophone in space
        # Theta / angle at measured pressure
        # Shape: (num of data samples, num of channels, num of signal samples, 1 or 2)
        self.space = None
        self.time = None
        self.pres = None
        self.theta = None
        self.space_time_pres_theta = None

        # subset to return
        self.sub_space_time_pres_theta = None
        # TODO:speeds
        #self._speeds = None
        
        # extract angels from dat files
        self._extract_signal_info()
        # extract taus from bin files
        self._extract_raw_signal()
        # geometry to space
        self._geometry_to_space()
        # pack data into 1
        self._pack_data()
        # set a upper and lower angles limit
        self.set_bound()


    # This function uses for sorting files
    def _key(self, file):
        # Sort by character at 26th place (digit)
        file_num = int(re.split('([0-9]+)',file)[1])
        return(file_num)
        
    def _sample_to_time(self, sampleFreq):
        # Convert sample to time
        # time per frequency sample
        time_per_sample = 1/sampleFreq
        # Get time stamp for pressuse u(t) by multiplying time per sample * sample[i]
        time_stamp = np.tile(np.arange(self._n_signal_samples), 
                             self.n_data_samples).reshape([self.n_data_samples,-1]) * time_per_sample[:,None]
        # Repeats for all channels and all data samples for convience
        # Shape: (num of data samples, num of channels, num of signal samples)
        self.time = np.tile(time_stamp,self._n_channels).reshape([self.n_data_samples,-1,self._n_signal_samples])

    def _geometry_to_space(self):
        # TODO: might be easier to use numpy tile instead
        # make a array of ones shape: (num of data samples, num of channels, dimensions)
        # dimensions => num of dimension of the array coordinate
        geo_dummy = np.ones([self.n_data_samples, *self.arr_geometry.shape], dtype=np.float32)
        # Each data sample has the same array geometry then add a dimension for num of signal samples
        # Shape: (num of data samples, num of channels, 1, dimensions)
        geo_expand = np.expand_dims(geo_dummy * self.arr_geometry, axis=2)
        # Each signal samples has the same array geometry
        # shape: (num of data samples, num of channels, num of signal samples, dimension)
        self.space = np.ones([self.n_data_samples, self._n_channels, self._n_signal_samples,
                              geo_expand.shape[-1]], dtype=np.float32) * geo_expand
        
    def _extract_signal_info(self):
        # Source signal information files
        files_dat = [i for i in os.listdir(self._path_dir) if os.path.isfile(os.path.join(self._path_dir,i)) and \
                 '.dat' in i]
        self._files_dat = sorted(files_dat, key = self._key)   
        
        # Extract signal information
        dats = []
        for file in self._files_dat:
            with open(self._path_dir+file, 'r') as f:
                lines = f.readlines()
                for i,l in enumerate(lines):
                    dat = [float(i) for i in l.split()]
                    dats.append(dat)
        # To numpy 
        dats = np.array(dats, dtype=np.float32)
        
        # number of data samples / size
        self.n_data_samples = len(dats)
        
        # Convert sample to time
        self._sample_to_time(dats[:,0])

        # Get theta
        angles = dats[:,2]
        # Repeat for all channels and all signal samples so each data samples shouldve the same theta for convience
        # Shape: (num of data samples, num of channels, num of signal samples)
        self.theta = np.repeat(angles, self._n_channels * 
                               self._n_signal_samples).reshape((self.n_data_samples, self._n_channels, -1))

    def _extract_raw_signal(self):
        # raw signals files
        files_raw = [i for i in os.listdir(self._path_dir) if os.path.isfile(os.path.join(self._path_dir,i)) and \
                           'raw.bin' in i]
        self._files_raw = sorted(files_raw, key = self._key)   
        
        # Extract signals
        signals = []
        for file in self._files_raw:
            with open(self._path_dir+file, 'rb') as f:
                signal = f.read()
                signal = np.frombuffer(signal, dtype = np.float32)
                signals.append(signal)
        signals_np = np.array(signals, dtype=np.float32)
        signals_np = signals_np.reshape(-1, self._n_signal_samples, self._n_channels)
        # Raw pressure u
        # Shape: (num of data samples, num of channels, num of signal samples)
        self.pres = np.matrix_transpose(signals_np)

    def _pack_data(self):
        # dimension
        space_dim = self.space.shape[-1]
        # pack each space, time, pressure and theta into a set
        self.space_time_pres_theta = np.empty([*self.time.shape, space_dim + 3], dtype=np.float32)
        self.space_time_pres_theta[:,:,:,:-3] = self.space
        self.space_time_pres_theta[:,:,:,-3] = self.time
        self.space_time_pres_theta[:,:,:,-2] = self.pres
        self.space_time_pres_theta[:,:,:,-1] = self.theta
        
    def set_bound(self, lower = 0.0, upper = 360.0):
        # get data in a bounded angles
        # since the theta is the same for all channels and signal in a data sample
        mask = np.logical_and(self.theta >= lower, self.theta <= upper)[:,0,0]
        self.sub_space_time_pres_theta = self.space_time_pres_theta[mask]

    def get_data(self, channels=None):
        # If no channels are specify
        # Default to channels from 0 to 23
        if not channels:
            channels = np.arange(0,24, dtype=int)
        return self.sub_space_time_pres_theta[:,channels]
        
    def plot(self, channels = None):
        if not channels:
            channels = list(range(0,self._n_channels))
        # Time delay vs angle at different channel
        fig = plt.figure(figsize = (20,20))
        ax = plt.axes()
        colors = cm.rainbow(np.linspace(0, 1, self._n_channels))
        for i in range(len(channels)):
            ax.scatter(self.angles, self.time_delays[:,channels[i]], label="ch{}".format(i), s=0.5,color=colors[channels[i]])
        plt.title("Time Delay vs. DOA for Each Channel")
        ax.grid()
        ax.set_xlabel('DOA (degree)', fontweight ='bold') 
        ax.set_ylabel('Time Delay (ms)', fontweight ='bold') 
        ax.legend(markerscale = 10)


In [3]:
class DataLoader:
    def __init__(self, space_time_pres_theta):
        # dataloader
        self.dataloader = None
        self.train_dataloader = None
        self.val_dataloader = None
        self.test_dataloader = None
        
        # Input for model: space and time
        self.space_time = None
        # Output for model: pressure and theta
        self.pres = None
        self.theta = None

        # the shape of data after permute: (num of data samples, num of signal samples, num of channels, (space, time, pressure, theta))
        # After reshape, each data point is a (space, time, pressure, theta)
        self._data = np.permute_dims(space_time_pres_theta, axes = (0,2,1,3))
        self._data = self._data.reshape([-1, self._data.shape[-2], self._data.shape[-1]])
        # numpy array to tensor and shuffle along the dimension 0
        self._data = tf.random.shuffle(self._data)

        # Unpacking the given data into separate arrays space, time, pressure, and theta
        self._unpack_data()

        # pack data into tensorflow dataloader
        self._pack_dataloader()

    def _unpack_data(self):
        # Unpacking the given data into separate arrays space, time, pressure, and theta
        # For space, space has different dimension
        # Since the time, pressure, and theta occupied the last 3 elements of each data point
        #self.space = self._data[:,:-3]
        #self.time = self._data[:,-3]
        self.space_time = self._data[:,:,:-2]
        self.pres = tf.expand_dims(self._data[:,:,-2], axis=-1)
        # since all channels has the same theta
        self.theta = tf.expand_dims(self._data[:,:,-1], axis=-1)
        
    def _pack_dataloader(self):
        # pack data into tensorflow dataloader
        #self.dataloader = tf.data.Dataset.from_tensor_slices(((self.space, self.time), (self.pres, self.theta)))
        self.dataloader = tf.data.Dataset.from_tensor_slices((self.space_time, self.pres, self.theta))
        
    def _shuffle_dataloader(self, buffer_size):
        # shuffle dataset
        if buffer_size == -1:
            self.dataloader = self.dataloader.shuffle(self.dataloader.cardinality())
        else:
            self.dataloader = self.dataloader.shuffle(buffer_size = buffer_size)
        
    def split(self, ratio=[0.7,0.15,0.15], shuffle=True, shuffle_buffer_size = 5):
        # return splits
        dataset_size = len(self._data)
        if shuffle:
            self._shuffle_dataloader(shuffle_buffer_size)
        if len(ratio) == 3:
            train_size = int(ratio[0]*dataset_size)
            val_size = int(ratio[1]*dataset_size)
            test_size = int(ratio[2]*dataset_size)
            self.train_dataloader = self.dataloader.take(train_size)
            self.test_dataloader = self.dataloader.skip(train_size)
            self.val_dataloader = self.dataloader.skip(test_size)
            self.test_dataloader = self.dataloader.take(test_size)
            return self.train_dataloader, self.val_dataloader, self.test_dataloader
        else:
            train_size = int(ratio[0]*dataset_size)
            test_size = int(ratio[1]*dataset_size)
            self.train_dataloader = self.dataloader.take(train_size)
            self.test_dataloader = self.dataloader.skip(train_size)
            return self.train_dataloader, self.test_dataloader


In [3]:
class Visualizer:
    def __init__(self, AA_geometry):
        self._AA_geometry_cart = AA_geometry
        self._AA_geometry_polar = np.empty(AA_geometry[:,:2].shape)
        self._AA_geometry_sphe = np.empty(AA_geometry.shape)
        self._cart2pol()
        self._cart2sphe()
    def cartesian2D(self, fig_size = (12,12)):
        fig = plt.figure(figsize = fig_size)
        ax = plt.axes()
        for i, channel in enumerate(self._AA_geometry_cart):
            ax.scatter(channel[0],channel[1], label = "ch{}".format(i))
            ax.text(channel[0], channel[1], '  %s'%(str(i)))
        plt.title("Array Cartesian Top View", pad = 25)
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlabel('X-axis', fontweight ='bold') 
        ax.set_ylabel('Y-axis', fontweight ='bold') 
        ax.margins(0.2)
        ax.legend()
        #plt.tight_layout()
    def cartesian3D(self, fig_size = (12,12)):
        fig = plt.figure(figsize = fig_size)
        ax = plt.axes(projection = "3d")
        for i, channel in enumerate(self._AA_geometry_cart):
            ax.scatter(channel[0],channel[1],channel[2], label = "ch{}".format(i))
            ax.text(channel[0], channel[1], channel[2], '  %s'%(str(i)), position=(1,1))
        plt.title("Array Cartesian 3D View", pad = 25)
        ax.set_xlabel('X-axis', fontweight ='bold') 
        ax.set_ylabel('Y-axis', fontweight ='bold') 
        ax.set_zlabel('Z-axis', fontweight ='bold')
        ax.legend()
    def polar(self, angle_1 = None, angle_2 = None, channels = [], fig_size = (15,14), save_dir = None):
        fig = plt.figure(figsize = fig_size)
        ax = plt.axes(projection = "polar")
        colors = cm.rainbow(np.linspace(0, 1, 24))
        for i, channel in enumerate(self._AA_geometry_polar):
            if i in channels:
                ax.scatter(channel[1],channel[0], s=300, marker="X", label = "ch{}".format(i), color=colors[i])
                ax.text(channel[1], channel[0], '  %s'%(str(i)) )
                continue
            ax.scatter(channel[1],channel[0], label = "ch{}".format(i), color=colors[i])
            ax.text(channel[1], channel[0], '  %s'%(str(i)) )
                
        if angle_1:
            angle = angle_1 * np.pi / 180
            ax.vlines(angle,0,0.12, colors = 'r')
        if angle_2:
            angle = angle_2 * np.pi / 180
            ax.vlines(angle,0,0.12, colors = 'g')
        plt.title("Array Cartesian 3D View", pad = 25)
        ax.margins(0.2)
        if save_dir:
            fig.savefig(save_dir+'AA_polar.png')
            
    def plot_dataset(self, dataset):
        input_dataset = []
        label_dataset = []
        for x,_,y in dataset:
            input_dataset.append(x)
            label_dataset.append(y)
        input_dataset = np.array(input_dataset)
        label_dataset = np.array(label_dataset)
        
        fig = plt.figure(figsize = (20,20))
        ax = plt.axes()
        for i in range(input_dataset.shape[1]):
            ax.scatter(label_dataset, input_dataset[:,[i]], label="ch{}".format(i))
        plt.title("Time Delay vs. DOA for Each Channel")
        ax.set_xlabel('DOA (degree)', fontweight ='bold') 
        ax.set_ylabel('Time Delay (ms)', fontweight ='bold') 
        ax.legend()
    def _cart2pol(self):
        for i, channel in enumerate(AA_Geometry):
            x = channel[0]
            y = channel[1]
            r = np.sqrt(x**2 + y**2)
            if x == 0:
                theta = np.pi/2 if y > 0 else -np.pi/2
            else:
                theta = np.arctan(y/x)# * 180 / np.pi
                theta = theta if x > 0 else theta + np.pi
            self._AA_geometry_polar[i][0] = r
            self._AA_geometry_polar[i][1] = theta
    def _cart2sphe(self):
        pass
        #for i in range(self._AA_geometry_cart.shape[1]):
        #    x = self._AA_geometry_cart[0][i] + 0.0001
        #    y = self._AA_geometry_cart[1][i] + 0.0001
        #    z = self._AA_geometry_cart[2][i] + 0.0001
        #    
        #    r = np.sqrt(x**2 + y**2 + z**2)
        #    theta = np.arctan(y/x)
        #    phi = np.arccos(z/r)
        #    self._AA_geometry_sphe[:,i] = np.array([r,theta,phi])
        


In [4]:
class EvaluateModel:
    def __init__(self, model, channel_id, dataset):
        self._model = model
        self._channel_id = channel_id
        self._dataset = dataset
    def plot_training(self, losses):
        metrics = list(losses.history.keys())
        fig, axs = plt .subplots(ncols = 3, nrows=1, figsize=(12,3), layout="constrained")
        for i in range(3):
            axs[i].plot(losses.history[metrics[i]], label="train_{}".format(metrics[i]))
            axs[i].plot(losses.history[metrics[i+3]], label=metrics[i+3])
            axs[i].set_yscale('log')
            axs[i].legend()
        fig.supxlabel("epoch")
        fig.suptitle("Channels: {}".format(" ".join(str(ch) for ch in self._channel_id)))

    def plot_evaluation(self, lower=0.0, upper=360.0, verbose = False, save_dir = None):
        fig, axs = plt.subplots(ncols = 1, nrows = len(self._channel_id) + 2, figsize = (20,len(self._channel_id)*3), layout="constrained")
        for x,c,y in self._dataset.batch(len(self._dataset)):
            # Targeting a specific range
            fillter = np.logical_and(y >= lower, y <= upper).flatten()
            x, y = x[fillter], y[fillter]
            y_pred, c_pred = self._model.predict(x, verbose=False)
            if verbose:
                print("Inputs")
                print(x)
                print("Y Truth", "Y Predict")
                print(np.stack((y, y_pred),axis=1).squeeze(-1))
            for i in range(x.shape[1]):
                axs[i].scatter(y, x[:,i], color='r', s=5)
                axs[i].scatter(y_pred, x[:,i], color='b', s=5)
                axs[i].set_title("Channel {}".format(self._channel_id[i]))
                axs[i].grid()

            # Absolute different
            axs[-2].scatter(y, y-y_pred, color='g', s=5)
            axs[-2].set_title("Truth vs Predict absolute different")
            axs[-2].grid()
            
            # Relative different
            y_diff = lambda y, y_hat : ((y - y_hat) + 180) % 360 -180
            axs[-1].scatter(y, y_diff(y, y_pred), color='g', s=5)
            axs[-1].set_title("Truth vs Predict relative different")
            axs[-1].grid()
        if save_dir:
            ch2str = ""
            for ch in channel_id:
                ch2str += "_{}".format(ch)
            fig.savefig(save_dir + "evaluation_ch{}.png".format(ch2str))
                            
    def evaluate(self, verbose=False):
        eva = self._model.evaluate(self._dataset.batch(100), verbose=False)
        if verbose:
            print(eva)
        return eva

In [5]:
def genRandomAA(num_channels, list_length):
    list_channels = np.empty([list_length, num_channels], dtype=int)
    for i in range(list_length):
        list_channels[i] = random.sample(range(0,24), num_channels)
    return list_channels

In [6]:
def evaluates_table(model, list_channels, evaluates):
    df = pd.DataFrame(np.zeros((len(list_channels), 5), dtype=object), columns=["Activation", "Channels", "Loss", "Mae", "Mse"])
    for i in range(len(list_channels)):
        df.at[i, "Activation"] = model
        df.at[i, "Channels"] = list_channels[i]
        df.at[i, "Loss"] = evaluates[i][0]
        df.at[i, "Mae"] = evaluates[i][2]
        df.at[i, "Mse"] = evaluates[i][1]
    return df

In [7]:
def plot_table(df, col_name, threshold=1):
    # Initialize matplot
    fig = plt.figure(figsize=(20,5))
    ax = plt.axes()
    # convert pandas dataframe to numpy
    target_values = df[col_name].to_numpy()
    # number of channels or length
    try:
        # case if the dataframe entries are in string
        num_channels = np.array([len(re.findall(r'\d+',item['Channels'])) for i,item in df.iterrows()])
    except:
        # case if the dataframe entries are object
        num_channels = np.array([len(item['Channels']) for i,item in df.iterrows()])
    mean = target_values.mean()
    std = target_values.std()
    # z scores
    zs = (target_values - mean) / std
    # Find the target values within the threshold if not mask with NaN
    plot_points = np.where(zs<threshold,target_values,np.nan)
    # Plot points vs number of channels
    ax.scatter(plot_points, num_channels, s = 5) 
    ax.grid(linestyle='--')
    ax.set_yticks(list(range(3,25,3)))
    ax.set_xlabel('Number of Channels', fontweight ='bold') 
    ax.set_ylabel(col_name, fontweight ='bold') 
    plt.title("Models Performance by " + col_name)